## Imports

In [1]:
import csv
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
rawData = []
with open('crude-oil-daily.csv', newline='') as csvfile:
  reader = csv.reader(csvfile)
  for row in reader:
    rawData.append(row)

In [3]:
df = pd.DataFrame(rawData)
df.columns = ['date', 'closingPx']
df = df.astype({'closingPx': 'float64', 'date': 'datetime64'})

In [4]:
df

,date,closingPx
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87
...,...,...
9349,2023-01-12,78.39
9350,2023-01-13,80.07
9351,2023-01-16,78.85
9352,2023-01-17,80.18


In [5]:
class MarketEqualizerGenerator():

  def __init__(self, df):
    self.df = df.copy()
    self.thresholds = []
    self.decay = [5, 3.75, 2.8, 2.1, 1.6, 1.18, 0.88, 0.66]
    self.threshold = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

  def generate(self):
    # Diffs
    self.df['diffFromPrev'] = np.exp(self.df.closingPx.diff())
    bucketDf = self.genBucketData()
    return pd.concat([self.df,bucketDf], axis=1)

  def genBucketData(self):
    # Set diff min to 0
    # TODO Do something with negatives?
    self.df.loc[self.df["diffFromPrev"] <= 0, "diffFromPrev"] = 0.0

    # Generate bucket data for each bucket
    bucketData = dict()
    for bucketN in range(8):
      delta = self.genBucketDelta(bucketN)
      bucketData[f'bucket{bucketN}'] = self.genBucketDataFromDelta(bucketN, delta)
    return pd.DataFrame(bucketData)

  def genBucketDelta(self, bucketN):
    bucketDelta = self.df.diffFromPrev.copy()
    bucketDelta.loc[bucketDelta < self.threshold[bucketN]] = 0.0
    return bucketDelta

  def genBucketDataFromDelta(self, bucketN, deltas):
    values = deltas.copy()
    values.values[:] = 0
    for index, delta in enumerate(deltas):
      if not math.isnan(delta):
        # Current value is previous value + delta - decay, floor to 0, ceil to 10
        values[index] = min(max(values[index-1] + delta - self.decay[bucketN], 0), 10)
    return values



    
  

In [6]:
meg = MarketEqualizerGenerator(df)
df = meg.generate()

In [7]:
df

,date,closingPx,diffFromPrev,bucket0,bucket1,bucket2,bucket3,bucket4,bucket5,bucket6,bucket7
0,1986-01-02,25.56,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1986-01-03,26.00,1.552707,0.000000,0.000000,0.000000,0.000000,0.000000,0.372707,0.672707,0.892707
2,1986-01-06,26.53,1.698932,0.000000,0.000000,0.000000,0.000000,0.098932,0.891640,1.491640,1.931640
3,1986-01-07,25.85,0.506617,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.611640,1.271640
4,1986-01-08,25.87,1.020201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.611640
...,...,...,...,...,...,...,...,...,...,...,...
9349,2023-01-12,78.39,1.973878,6.973878,8.223878,9.173878,9.873878,10.000000,10.000000,10.000000,10.000000
9350,2023-01-13,80.07,5.365556,7.339434,9.839434,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
9351,2023-01-16,78.85,0.295230,2.634664,6.384664,7.200000,7.900000,8.400000,8.820000,9.120000,9.340000
9352,2023-01-17,80.18,3.781043,1.415707,6.415707,8.181043,9.581043,10.000000,10.000000,10.000000,10.000000


In [8]:
colNames = ["bucket0", "bucket1", "bucket2", "bucket3", "bucket4", "bucket5", "bucket6", "bucket7"]

for index, row in tqdm(df.iterrows()):
  fig, axes = plt.subplots(nrows=2, ncols=1)

  axes[0].bar(range(8),row[colNames], color=['black', 'red', 'green', 'blue', 'cyan'])
  axes[0].set_ylim(0, 10)

  df.plot.line(x='date', y='closingPx', ax=axes[1])
  axes[1].axvline(row.date, color='k', linestyle='--')

  plt.close(fig)
  fig.savefig(f'figs/{index}.png')

3501it [08:12,  7.99it/s]